<a href="https://colab.research.google.com/github/lisethrubio/Project4-Machine_Learning_Model/blob/Steven_branch2/bodyPerformance_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.2'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [57.5 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,150 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,267 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:14 https

In [7]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [8]:
# Read in data from bodyPerformance.csv
from pyspark import SparkFiles
csv = "/content/bodyPerformance.csv"
spark.sparkContext.addFile(csv)
df = spark.read.csv(SparkFiles.get("bodyPerformance.csv"), sep=",", header=True)
df.show()

+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
| age|gender|height_cm|weight_kg|body fat_%|diastolic|systolic|gripForce|sit and bend forward_cm|sit-ups counts|broad jump_cm|class|
+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
|27.0|     M|    172.3|    75.24|      21.3|     80.0|   130.0|     54.9|                   18.4|          60.0|        217.0|    C|
|25.0|     M|    165.0|     55.8|      15.7|     77.0|   126.0|     36.4|                   16.3|          53.0|        229.0|    A|
|31.0|     M|    179.6|     78.0|      20.1|     92.0|   152.0|     44.8|                   12.0|          49.0|        181.0|    C|
|32.0|     M|    174.5|     71.1|      18.4|     76.0|   147.0|     41.4|                   15.2|          53.0|        219.0|    B|
|28.0|     M|    173.8|     67.7|      17.1|     70.0|   127.0|     4

In [9]:
# Restructure class column into 0 and 1
df = df.withColumn('class', when(col('class').isin('A', 'B'), 0).otherwise(1))

In [10]:
# Rename the column to remove spaces and special characters
df = df.withColumnRenamed("body fat_%", "body_fat_percent") \
       .withColumnRenamed("sit and bend forward_cm", "sit_and_bend_forward_cm") \
       .withColumnRenamed("sit-ups counts", "sit_ups_counts") \
       .withColumnRenamed("broad jump_cm", "broad_jump_cm")
df.show()

+----+------+---------+---------+----------------+---------+--------+---------+-----------------------+--------------+-------------+-----+
| age|gender|height_cm|weight_kg|body_fat_percent|diastolic|systolic|gripForce|sit_and_bend_forward_cm|sit_ups_counts|broad_jump_cm|class|
+----+------+---------+---------+----------------+---------+--------+---------+-----------------------+--------------+-------------+-----+
|27.0|     M|    172.3|    75.24|            21.3|     80.0|   130.0|     54.9|                   18.4|          60.0|        217.0|    1|
|25.0|     M|    165.0|     55.8|            15.7|     77.0|   126.0|     36.4|                   16.3|          53.0|        229.0|    0|
|31.0|     M|    179.6|     78.0|            20.1|     92.0|   152.0|     44.8|                   12.0|          49.0|        181.0|    1|
|32.0|     M|    174.5|     71.1|            18.4|     76.0|   147.0|     41.4|                   15.2|          53.0|        219.0|    0|
|28.0|     M|    173.8|    

In [11]:
# Create a temporary view
df.createOrReplaceTempView('health')

In [12]:
# Run a sql query that groups the data on class, showing body fat and blood pressure
start_time = time.time()

spark.sql("""select class, avg(body_fat_percent), avg(diastolic), avg(systolic) from health group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+---------------------+-----------------+------------------+
|class|avg(body_fat_percent)|   avg(diastolic)|     avg(systolic)|
+-----+---------------------+-----------------+------------------+
|    1|   25.189998143027772|79.31360107494774|130.50492684383397|
|    0|   21.289458047050044| 78.2798506348021|129.96458551157582|
+-----+---------------------+-----------------+------------------+

--- 3.3236751556396484 seconds ---


In [13]:
# Run a sql query that groups the data on class, showing exercise scores
start_time = time.time()

spark.sql("""select class, avg(gripForce), avg(sit_and_bend_forward_cm), avg(sit_ups_counts), avg(broad_jump_cm) from health group by 1""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+-----------------+----------------------------+-------------------+------------------+
|class|   avg(gripForce)|avg(sit_and_bend_forward_cm)|avg(sit_ups_counts)|avg(broad_jump_cm)|
+-----+-----------------+----------------------------+-------------------+------------------+
|    1|35.66520453866825|          10.990452373842926|   34.3007763511496| 181.2243953418931|
|    0|38.26313218820005|           19.42997460791637|  45.24412247946229| 199.0388498879761|
+-----+-----------------+----------------------------+-------------------+------------------+

--- 1.8406341075897217 seconds ---
